<a href="https://colab.research.google.com/github/fateme-hshm96/UU6966-TTM/blob/main/fewshotT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [4]:
!pip install --quiet transformers==4.1.1
!pip install --quiet pytorch-lightning==1.1.3
!pip install --quiet tokenizers==0.9.4 
!pip install --quiet sentencepiece==0.1.94
!pip install --quiet tqdm==4.56.0

     |████████████████████████████████| 1.5 MB 4.7 MB/s 
     |████████████████████████████████| 2.9 MB 52.9 MB/s 
     |████████████████████████████████| 880 kB 65.1 MB/s 
     |████████████████████████████████| 680 kB 4.8 MB/s 
     |████████████████████████████████| 829 kB 64.4 MB/s 
     |████████████████████████████████| 1.1 MB 4.9 MB/s 
     |████████████████████████████████| 72 kB 1.0 MB/s 


In [5]:
!pip install --quiet --upgrade pytorch-lightning

     |████████████████████████████████| 796 kB 4.6 MB/s 
     |████████████████████████████████| 529 kB 58.9 MB/s 
     |████████████████████████████████| 136 kB 71.6 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 87 kB 8.5 MB/s 


In [ ]:
# restart runtime - make sure insatlled packages and libraries are recognized by the system
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [1]:
# Check we have a GPU and check the memory size of the GUP
!nvidia-smi

Thu Nov 10 17:21:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Model Preparation

In [2]:
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

pl.seed_everything(42)

INFO:lightning_lite.utilities.seed:Global seed set to 42


42

In [3]:
# model_name = 't5-base'
model_name = 'google/flan-t5-xl'
t5_tokenizer = T5Tokenizer.from_pretrained(model_name) 
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# Datasets

# What if
regenerate_answer_triples_train = [
    ("what would happen to the predictions for instances with diabetes pedigree function less than 0.368 if you were to change bmi to 22.3?","diabetes is unlikely 88%","For this subset of instances with pedigree function less than 0.368, if bmi is 22.3, then 88% are unlikely to have diabetes and the rest are likely to have it."),
    ("what would happen to the class probabilities if we increased blood pressure by 94?","diabetes is unlikely 80%","Increasing blood pressure by 94 will put 80% of the data into the class of unlikely to have diabetes."),
    ("if we took 40 points away from skin thickness for id 157 what would happen to the probability of the predictions?","diabetes is likely by 80% probability","With this change in skin thickness, patient 157 is 80% likey to have diabetes."),
    ("what would the predictions be if you set diabetes pedigree function to 0.409 on the entire dataset?","35% likely to have diabetes","Setting the diabetes pedigree function to 0.409 causes 35% of instances in the entire dataset likely to have diabetes."),
    ("how likely are people with glucose below 166 and at least 22.3 bmi to be unlikely to have diabetes","31% are unliekly to have diabetes","31% of people with glucose below 166 and at least 22.3 bmi are unlikely to have diabetes"),
    ("probability data point 41 is predicted by the model unlikely to have diabetes","63%","patient 41 is unlikely to have diabetes with a probability of 63%")
]

regenerate_answer_triples_validation = [
    ("tell me what the model would predict if you decreased skin thickness by 5 on all the data","47 percent likely to have diabetes","decreasing the skin thickness by 5 makes 47% of instances in the entire dataset likely to have diabetes."),
    ("for people with a bmi over 38.9 or pregnancies over 1, how likely are they to be unlikely to have diabetes?","for 20% diabetes is unlikely","20% of the people with a bmi over 38.9 or pregnancies over 1 are likely to have diabetes")            
]

# Feature importance
# regenerate_answer_triples_train = [
    # ("could you please indicate when pregnancies is important?","pregnancies ranked 6","pregnancies is a somewhat important feature with rank 6."),
    # ("most important feature?","glucose","glucose is the most important feature at rank 1."),
    # ("tell me second most important feature for men","bmi","for men bmi is the second most important feature."),
    # ("what is the third most important feature?","age","age is the third most important feature"),
    # ("if people with glucose is not greater than 130 were to have pregnancies decreased by 6, what would the top 5 most important features be?","1 glucose, 2 bmi, 3 age, 4 diabetes pedigree function, 5 blood pressure","for data instances with glucose level less than or equal to 130, if pregnancies is decreased by 6, 5 most important features are as follows 1 glucose, 2 bmi, 3 age, 4 diabetes pedigree function, 5 blood pressure"),
    # ("5 most important features for blood pressure equal to 94","1 diabetes pedigree function, 2 glucose, 3 age, 4 bmi, 5 blood pressure","For patients with blood pressure equal to 94 important features are diabetes pedigree function, glucose, age, bmi, blood pressure.")
# ]

In [5]:
from tqdm.notebook import tqdm
import copy


class RegenerateAnswerDataste(Dataset):
    def __init__(self, tokenizer, tf_list, max_len_inp=96,max_len_out=96):

        self.regenerate_answer_triples = tf_list

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.skippedcount =0
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for inputs,content,outputs in self.regenerate_answer_triples:
          input_sent = "question: "+inputs+"\ninformation: "+content
          ouput_sent = "answer: "+outputs

          # tokenize inputs
          tokenized_inputs = self.tokenizer.batch_encode_plus(
              [input_sent], max_length=self.max_len_input, pad_to_max_length=True, return_tensors="pt"
          )
          # tokenize targets
          tokenized_targets = self.tokenizer.batch_encode_plus(
              [ouput_sent], max_length=self.max_len_output, pad_to_max_length=True,return_tensors="pt"
          )

          self.inputs.append(tokenized_inputs)
          self.targets.append(tokenized_targets)


In [18]:
train_dataset = RegenerateAnswerDataste(t5_tokenizer,regenerate_answer_triples_train)
validation_dataset = RegenerateAnswerDataste(t5_tokenizer,regenerate_answer_triples_validation)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [9]:
class T5FineTuner(pl.LightningModule):
    def __init__(self,hparams, t5model, t5tokenizer):
        super(T5FineTuner, self).__init__()
        # self.hparams = hparams
        for key in hparams.keys():
          self.hparams[key]=hparams[key]
        self.model = t5model
        self.tokenizer = t5tokenizer


    def forward( self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )
         
         return outputs


    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.hparams.batch_size,num_workers=4)

    def val_dataloader(self):
        return DataLoader(validation_dataset, batch_size=self.hparams.batch_size,num_workers=4)



    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer


Train

In [19]:
import argparse
args_dict = dict(
    batch_size=1,
)

# args = argparse.Namespace(**args_dict)
# print(args)

model = T5FineTuner(args_dict, t5model=t5_model,t5tokenizer=t5_tokenizer)

trainer = pl.Trainer(max_epochs = 5, gpus=1) #, progress_bar_refresh_rate=30)

trainer.fit(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1559: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Test

In [25]:
# test_sent = 'question: what is the chance that the data point with id 595 is likely to have diabetes?\ninformation: 79%</s>'  # what if
# test_sent = "question: tell me what the model would predict if you decreased skin thickness by 5 on all the data\ninformation: for 64 percent diabetes is unlikely</s>"  # what it
test_sent = "question: what are the class probabilities on your predictions for diabetes pedigree function more than 0.409?\ninformation: 52% unlikely</s>"  # pred likelihood
# test_sent = "question: third most important feature?\ninformation: skin thickness"
# test_sent = "flip the classification for sample 595?\ninformation: is likely. decrease glucose to 49.0</s>" # change pred

test_tokenized = t5_tokenizer.encode_plus(test_sent, return_tensors="pt")
test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

model.model.eval()
beam_outputs = model.model.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=5,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = t5_tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(sent)


answer: 52% of people with pedigree function more than 0.409 are unlikely to have diabetes.
answer: 52% of patients with pedigree function more than 0.409 are unlikely to have diabetes.
answer: 52% of people with a pedigree function more than 0.409 are unlikely to have diabetes.
answer: 52% of people with pedigree function more than 0.409 are unlikely to have diabetes
answer: 52% of patients with a pedigree function more than 0.409 are unlikely to have diabetes.
